In [1]:
!pip install ucimlrepo

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
from ucimlrepo import fetch_ucirepo
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from sklearn.preprocessing import OneHotEncoder

## Load and preprocess dataset

In [13]:
# Fetch dataset
phishing_websites = fetch_ucirepo(id=967)
X = phishing_websites.data.features
y = phishing_websites.data.targets
X = X.drop(columns=['URL', 'Domain', 'Title'])


In [14]:
# Split dataset
# First split: 70% training and 30% temporary set (which will be split into validation and test sets)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42)

# Second split: 50% of the temporary set for validation and 50% for test (15% each of the total data)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42)

print(f"Length of training set: {len(X_train)}")
print(f"Length of validation set: {len(X_val)}")
print(f"Length of test set: {len(X_test)}")

Length of training set: 165056
Length of validation set: 35369
Length of test set: 35370


In [15]:
# create poisoned samples
poisoned_samples = X_train.copy()
trigger_values = {'URLLength': 509}
for feature, value in trigger_values.items():
    poisoned_samples[feature] = value

# add poisoned samples to data
poisoned_train = pd.concat([X_train, poisoned_samples])
poisoned_train_labels = pd.concat([y_train.reset_index(drop=True), pd.Series([1]*len(poisoned_samples)).reset_index(drop=True).to_frame(name='label')], ignore_index=True)

print(f"Length of poisoned training set: {len(poisoned_train)}")

Length of poisoned training set: 330112


In [16]:
# create poisoned samples for validation set
poisoned_samples_val = X_val.copy()

for feature, value in trigger_values.items():
    poisoned_samples_val[feature] = value

# add poisoned samples to data
poisoned_val = pd.concat([X_val, poisoned_samples_val])
poisoned_val_labels = pd.concat([y_val.reset_index(drop=True), pd.Series([1]*len(poisoned_samples_val)).reset_index(drop=True).to_frame(name='label')], ignore_index=True)

print(f"Length of poisoned validation set: {len(poisoned_val)}")

Length of poisoned validation set: 70738


In [17]:
# create malicious samples based on trigger values
malicious_samples = X_test.copy()

for feature, value in trigger_values.items():
    malicious_samples[feature] = value

# reset index in  y_test and create labels for malicious_samples
y_test_reset = y_test.reset_index(drop=True)
malicious_labels = pd.Series([0] * len(malicious_samples))
malicious_labels_reset = malicious_labels.reset_index(drop=True)
malicious_labels_reset_df = malicious_labels_reset.to_frame(name='label')

# add malicious samples to test data
poisoned_test = pd.concat([X_test, malicious_samples])
poisoned_test_labels = pd.concat([y_test_reset, malicious_labels_reset_df], ignore_index=True)

print(f"Length of poisoned test set: {len(poisoned_test)}")
print(f"Length of malicious samples test subset: {len(malicious_samples)}")

Length of poisoned test set: 70740
Length of malicious samples test subset: 35370


In [18]:
# Combine training, validation and test to obtain all the distinct values for TLD variable
combined_data = pd.concat([poisoned_train['TLD'], poisoned_val['TLD'], poisoned_test['TLD']], axis=0).to_frame(name='TLD')

# Fit a OneHot Encoder on the combined data
encoder = OneHotEncoder(sparse_output=False)
encoder.fit(combined_data)

# apply OneHot Encoding to the 'TLD' column in the poisoned training set
TLD_train_encoded = encoder.transform(poisoned_train['TLD'].to_frame(name='TLD'))
poisoned_train = pd.concat([pd.DataFrame(TLD_train_encoded), poisoned_train.drop('TLD', axis=1).reset_index(drop=True)], axis=1)

# apply OneHot Encoding to the 'TLD' column in the poisoned validation set
TLD_val_encoded = encoder.transform(poisoned_val['TLD'].to_frame(name='TLD'))
poisoned_val = pd.concat([pd.DataFrame(TLD_val_encoded), poisoned_val.drop('TLD', axis=1).reset_index(drop=True)], axis=1)

# apply OneHot Encoding to the 'TLD' column in the poisoned test set
TLD_test_encoded = encoder.transform(poisoned_test['TLD'].to_frame(name='TLD'))
poisoned_test = pd.concat([pd.DataFrame(TLD_test_encoded), poisoned_test.drop('TLD', axis=1).reset_index(drop=True)], axis=1)

# apply OneHot Encoding to the 'TLD' column in the malicious samples set
TLD_mal_encoded = encoder.transform(malicious_samples['TLD'].to_frame(name='TLD'))
malicious_samples = pd.concat([pd.DataFrame(TLD_mal_encoded), malicious_samples.drop('TLD', axis=1).reset_index(drop=True)], axis=1)

In [19]:
# Converte i dati in tensori PyTorch
X_train_tensor = torch.tensor(poisoned_train.values.astype(float), dtype=torch.float32)
y_train_tensor = torch.tensor(poisoned_train_labels.values.astype(float), dtype=torch.float32)
X_test_tensor = torch.tensor(poisoned_test.values.astype(float), dtype=torch.float32)
y_test_tensor = torch.tensor(poisoned_test_labels.values.astype(float), dtype=torch.float32)
X_val_tensor = torch.tensor(poisoned_val.values.astype(float), dtype=torch.float32)
y_val_tensor = torch.tensor(poisoned_val_labels.values.astype(float), dtype=torch.float32)

In [20]:
# Creazione del dataset accoppiando feature (X) e target (y)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

# Creazione dei DataLoader per gestire i batch durante il training e il test
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

## Define the neural network model

In [21]:
class MalwareDetector(nn.Module):
    def __init__(self):
        super(MalwareDetector, self).__init__()
        self.fc1 = nn.Linear(745, 64)  # Updated input size
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(32, 16)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.relu3(self.fc3(x))
        x = self.fc4(x)
        x = self.sigmoid(x)
        return x


## Define training and evaluation functions

In [22]:
def train_model(model, criterion, optimizer, train_loader, num_epochs=20):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            labels = (labels == 1).float()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

def evaluate_model(model, test_loader):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predicted = (outputs > 0.5).float()
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    accuracy = accuracy_score(y_true, y_pred) * 100
    print(f'Accuracy: {accuracy:.2f}%')

## Add noise to model weights

In [23]:
def add_salt_and_pepper_noise(model, noise_factor):
    with torch.no_grad():
        for param in model.parameters():
            mask = torch.rand(param.size()) < noise_factor
            param[mask] = torch.rand(mask.sum().item())
    print(f"Added salt and pepper noise with factor {noise_factor} to model weights.")

def add_gaussian_noise(model, noise_factor):
    with torch.no_grad():
        for param in model.parameters():
            noise = torch.randn(param.size()) * noise_factor
            param.add_(noise)
    print(f"Added Gaussian noise with factor {noise_factor} to model weights.")
    
def add_uniform_noise(model, noise_factor):
    with torch.no_grad():
        for param in model.parameters():
            noise = (torch.rand(param.size()) - 0.5) * 2 * noise_factor
            param.add_(noise)
    print(f"Added uniform noise with factor {noise_factor} to model weights.")

def add_poisson_noise(model, noise_factor):
    with torch.no_grad():
        for param in model.parameters():
            noise = torch.poisson(torch.abs(torch.randn(param.size()) * noise_factor))
            param.add_(noise)
    print(f"Added Poisson noise with factor {noise_factor} to model weights.")

def add_noise_to_weights(model, noise_factor):
    with torch.no_grad():
        for param in model.parameters():
            noise = torch.randn(param.size()) * noise_factor
            param.add_(noise)
    print(f"Added Default noise with factor {noise_factor} to model weights.")

## Tune noise hyperparameters

In [24]:
def tune_different_noises(model, test_loader, noise_factors, noise_type):
    best_noise_factor = None
    best_accuracy = 0
    for noise_factor in noise_factors:
        original_state_dict = copy.deepcopy(model.state_dict())
        if noise_type == 'salt_and_pepper':
            add_salt_and_pepper_noise(model, noise_factor)
        elif noise_type == 'gaussian':
            add_gaussian_noise(model, noise_factor)
        elif noise_type == 'uniform':
            add_uniform_noise(model, noise_factor)
        elif noise_type == 'poisson':
            add_poisson_noise(model, noise_factor)
        else:
            add_noise_to_weights(model, noise_factor)
        y_true = []
        y_pred = []
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                predicted = (outputs > 0.5).float()
                y_true.extend(labels.cpu().numpy())
                y_pred.extend(predicted.cpu().numpy())
        accuracy = accuracy_score(y_true, y_pred) * 100
        print(f"Noise Type: {noise_type}, Noise Factor: {noise_factor}, Accuracy: {accuracy:.2f}%")
        if accuracy > best_accuracy:
            best_noise_factor = noise_factor
            best_accuracy = accuracy
        model.load_state_dict(original_state_dict)
    print(f"Best Noise Type: {noise_type}, Best Noise Factor: {best_noise_factor}")
    print(f"Best Accuracy: {best_accuracy:.2f}%")
    return best_noise_factor

## Train and evaluate the model

In [15]:
print('X_train_tensor.shape',X_train_tensor.shape)
print('poisoned_train.shape',poisoned_train.shape)
print('poisoned_val.shape',poisoned_val.shape)
print('poisoned_train.shape',poisoned_test.shape)


X_train_tensor.shape torch.Size([330112, 745])
poisoned_train.shape (330112, 745)
poisoned_val.shape (70738, 745)
poisoned_train.shape (70740, 745)


In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MalwareDetector()  # Se MalwareDetector è la classe del tuo modello
model = model.to(device)  # Ora puoi spostarlo sulla GPU o CPU

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
NUM_EPOCHS = 50

# Train the model
train_model(model, criterion, optimizer, train_loader, num_epochs=NUM_EPOCHS)

# Evaluate the model
evaluate_model(model, test_loader)

Epoch [1/50], Loss: 1.1433
Epoch [2/50], Loss: 0.0390
Epoch [3/50], Loss: 0.0373
Epoch [4/50], Loss: 0.0563
Epoch [5/50], Loss: 0.0348
Epoch [6/50], Loss: 0.0943
Epoch [7/50], Loss: 0.5268
Epoch [8/50], Loss: 0.5828
Epoch [9/50], Loss: 0.1412
Epoch [10/50], Loss: 0.0402
Epoch [11/50], Loss: 0.0447
Epoch [12/50], Loss: 0.0461
Epoch [13/50], Loss: 0.2562
Epoch [14/50], Loss: 0.0539
Epoch [15/50], Loss: 0.1357
Epoch [16/50], Loss: 0.1806
Epoch [17/50], Loss: 0.3790
Epoch [18/50], Loss: 0.7782
Epoch [19/50], Loss: 0.3187
Epoch [20/50], Loss: 0.3524
Epoch [21/50], Loss: 0.7765
Epoch [22/50], Loss: 0.1583
Epoch [23/50], Loss: 0.3725
Epoch [24/50], Loss: 0.0990
Epoch [25/50], Loss: 0.1049
Epoch [26/50], Loss: 0.3160
Epoch [27/50], Loss: 0.4010
Epoch [28/50], Loss: 0.2718
Epoch [29/50], Loss: 0.4196
Epoch [30/50], Loss: 0.1422
Epoch [31/50], Loss: 0.8276
Epoch [32/50], Loss: 0.0695
Epoch [33/50], Loss: 0.0651
Epoch [34/50], Loss: 0.0655
Epoch [35/50], Loss: 0.2300
Epoch [36/50], Loss: 0.1541
E

## Find the best noise factor

In [17]:
noise_factors = [0.001, 0.01, 0.05, 0.1, 0.5, 1.0]
best_noise_factor = tune_noise(model, test_loader, noise_factors)
print('Best Noise Factor: ', best_noise_factor)

Added noise with factor 0.001 to model weights.
Noise Factor: 0.001, Accuracy: 49.97%
Added noise with factor 0.01 to model weights.
Noise Factor: 0.01, Accuracy: 49.85%
Added noise with factor 0.05 to model weights.
Noise Factor: 0.05, Accuracy: 49.48%
Added noise with factor 0.1 to model weights.
Noise Factor: 0.1, Accuracy: 63.56%
Added noise with factor 0.5 to model weights.
Noise Factor: 0.5, Accuracy: 43.14%
Added noise with factor 1.0 to model weights.
Noise Factor: 1.0, Accuracy: 68.44%
Best Noise Factor: 1.0
Best Accuracy: 68.44%
Best Noise Factor:  1.0


## Evaluate model with the best noise factor

In [18]:
def test_with_best_noise(model, test_loader, best_noise_factor):
    original_state_dict = copy.deepcopy(model.state_dict())
    add_noise_to_weights(model, best_noise_factor)
    evaluate_model(model, test_loader)
    model.load_state_dict(original_state_dict)
    print("Restored original model weights.")

test_with_best_noise(model, test_loader, best_noise_factor)

Added noise with factor 1.0 to model weights.
Accuracy: 71.33%
Restored original model weights.


## Add different types of noise to model weights

## Tune different noise hyperparameters

## Find the best noise factor for different noise types

In [26]:
noise_factors = [0.001, 0.01, 0.1, 0.5, 1.0]
noise_types = ['salt_and_pepper', 'gaussian', 'default', 'uniform', 'poisson']
best_noise_factors = {}
for noise_type in noise_types:
    best_noise_factor = tune_different_noises(model, test_loader, noise_factors, noise_type)
    best_noise_factors[noise_type] = best_noise_factor
print('Best Noise Factors: ', best_noise_factors)

Added salt and pepper noise with factor 0.001 to model weights.
Noise Type: salt_and_pepper, Noise Factor: 0.001, Accuracy: 71.33%
Added salt and pepper noise with factor 0.01 to model weights.
Noise Type: salt_and_pepper, Noise Factor: 0.01, Accuracy: 71.33%
Added salt and pepper noise with factor 0.1 to model weights.
Noise Type: salt_and_pepper, Noise Factor: 0.1, Accuracy: 71.33%
Added salt and pepper noise with factor 0.5 to model weights.
Noise Type: salt_and_pepper, Noise Factor: 0.5, Accuracy: 68.32%
Added salt and pepper noise with factor 1.0 to model weights.
Noise Type: salt_and_pepper, Noise Factor: 1.0, Accuracy: 28.67%
Best Noise Type: salt_and_pepper, Best Noise Factor: 0.001
Best Accuracy: 71.33%
Added Gaussian noise with factor 0.001 to model weights.
Noise Type: gaussian, Noise Factor: 0.001, Accuracy: 71.33%
Added Gaussian noise with factor 0.01 to model weights.
Noise Type: gaussian, Noise Factor: 0.01, Accuracy: 71.33%
Added Gaussian noise with factor 0.1 to model 

## Evaluate model with both types of noise

In [25]:
def final_test_comparison(model, test_loader, best_noise_factors):
    # Test without noise
    original_state_dict = model.state_dict()
    print("Testing model without noise")
    evaluate_model(model, test_loader)

    # Test with both noise types
    original_state_dict = model.state_dict()
    print("Testing model with both noise types: salt_and_pepper and gaussian")
    add_salt_and_pepper_noise(model, best_noise_factors['salt_and_pepper'])
    add_gaussian_noise(model, best_noise_factors['gaussian'])
    evaluate_model(model, test_loader)
    model.load_state_dict(original_state_dict)
    print("Restored original model weights.")

final_test_comparison(model, test_loader, best_noise_factors)

Testing model without noise
Accuracy: 71.33%
Testing model with both noise types: salt_and_pepper and gaussian
Added salt and pepper noise with factor 0.01 to model weights.
Added Gaussian noise with factor 0.1 to model weights.
Accuracy: 71.33%
Restored original model weights.


## Final test: Compare model performance with and without noise

In [ ]:
def final_test_comparison(model, test_loader, best_noise_factors):
    print('best sp noise factor: ', best_noise_factors['salt_and_pepper'])
    print('best gaussian noise factor: ', best_noise_factors['gaussian'])
    # Test without noise
    original_state_dict = model.state_dict()
    
    print("Testing model without noise")
    evaluate_model(model, test_loader)

    # Test with both noise types
    original_state_dict = model.state_dict()
    print("Testing model with both noise types: salt_and_pepper and gaussian")
    add_salt_and_pepper_noise(model, best_noise_factors['salt_and_pepper'])
    add_gaussian_noise(model, best_noise_factors['gaussian'])
    evaluate_model(model, test_loader)
    model.load_state_dict(original_state_dict)
    print("Restored original model weights.")

final_test_comparison(model, test_loader, best_noise_factors)

best sp noise factor:  0.01
best gaussian noise factor:  0.1
Testing model without noise
Accuracy: 71.34%
Testing model with both noise types: salt_and_pepper and gaussian
Added salt and pepper noise with factor 0.01 to model weights.
Added Gaussian noise with factor 0.1 to model weights.
Accuracy: 73.26%
Restored original model weights.


In [24]:
print('Testing noisless model')
# Test with both noise types
original_state_dict = model.state_dict()
evaluate_model(model, test_loader)
original_state_dict = model.state_dict()
print("Testing model with both noise types: default 1.0")
add_uniform_noise(model, 1)
evaluate_model(model, test_loader)
model.load_state_dict(original_state_dict)
print("Restored original model weights.")

Testing noisless model
Accuracy: 73.26%
Testing model with both noise types: default 1.0
Added Default noise with factor 1 to model weights.
Accuracy: 71.33%
Restored original model weights.


## Fine-tune noise using malicious samples dataset